In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
# import training data
dft_eng = pd.read_csv('../data/dft_eng.csv')
dft_jap = pd.read_csv('../data/dft_jap.csv')
dft_fin = pd.read_csv('../data/dft_fin.csv')

# import validation data
dfv_eng = pd.read_csv('../data/dfv_eng.csv')
dfv_jap = pd.read_csv('../data/dfv_jap.csv')
dfv_fin = pd.read_csv('../data/dfv_fin.csv')

#import word count
word_count = pd.read_csv('../data/question_word_count.csv')

In [5]:
def ans_freq(que, doc):

    freq = [x for x in que if x in doc]
    freq = len(freq)/len(que)
    return freq

In [6]:
# calculating frequenzy of words in answer that are in text 
for df in [dft_jap,dfv_jap,dft_eng,dfv_eng,dft_fin,dfv_fin]:
    frequency = []
    for question, answer in zip(df['question_text_tokenized'], df['document_plaintext_tokenized']):
        frequency.append(ans_freq(eval(question), eval(answer)))

    df['word_frequency_score'] = frequency

In [7]:
dft_eng

,question_text,document_title,language,annotations,document_plaintext,document_url,answer_start,answer_text,question_text_tokenized,document_plaintext_tokenized,answer_text_tokenized,labels,word_frequency_score
0,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': array([159]), 'answer_text': ...",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,[159],['1920s'],"['when', 'was', 'quantum', 'field', 'theory', ...","['quantum', 'field', 'theory', 'naturally', 'b...",['1920s'],1,0.571429
1,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': array([610]), 'answer_text': ...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,[610],['Sully Prudhomme'],"['who', 'was', 'the', 'first', 'nobel', 'prize...","['the', 'nobel', 'prize', 'in', 'literature', ...","['sully', 'prudhomme']",1,0.700000
2,When is the dialectical method used?,Dialectic,english,"{'answer_start': array([129]), 'answer_text': ...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,[129],['discourse between two or more people holding...,"['when', 'is', 'the', 'dialectical', 'method',...","['dialectic', 'or', 'dialectics', '(', 'greek'...","['discourse', 'between', 'two', 'or', 'more', ...",1,0.571429
3,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': array([88]), 'answer_text': a...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,[88],['Sejong the Great'],"['who', 'invented', 'hangul', '?']","['hangul', 'was', 'personally', 'created', 'an...","['sejong', 'the', 'great']",1,0.250000
4,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': array([0]), 'answer_text': ar...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,[0],"['Grasshoppers are plant-eaters, with a few sp...","['what', 'do', 'grasshoppers', 'eat', '?']","['grasshoppers', 'are', 'plant-eaters', ',', '...","['grasshoppers', 'are', 'plant-eaters', ',', '...",1,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7384,What was Neil Brooks' fastest recorded time?,Swimming at the 1980 Summer Olympics – Men's 4...,english,"{'answer_start': array([-1]), 'answer_text': a...",The medley relay was scheduled in the Olympisk...,https://en.wikipedia.org/wiki/Swimming%20at%20...,[-1],[''],"['what', 'was', 'neil', 'brooks', ""'"", 'fastes...","['the', 'medley', 'relay', 'was', 'scheduled',...",[],0,0.444444
7385,Who are the three most important eastern philo...,Eastern philosophy,english,"{'answer_start': array([-1]), 'answer_text': a...",Sāmkhya is a dualist philosophical tradition b...,https://en.wikipedia.org/wiki/Eastern%20philos...,[-1],[''],"['who', 'are', 'the', 'three', 'most', 'import...","['sāmkhya', 'is', 'a', 'dualist', 'philosophic...",[],0,0.222222
7386,Who was costume designer for the first Star Wa...,John Mollo,english,"{'answer_start': array([-1]), 'answer_text': a...",Mollo was surprised by the success of Star War...,https://en.wikipedia.org/wiki/John%20Mollo,[-1],[''],"['who', 'was', 'costume', 'designer', 'for', '...","['mollo', 'was', 'surprised', 'by', 'the', 'su...",[],0,0.545455
7387,Who developed the first thermonuclear weapon?,History of nuclear weapons,english,"{'answer_start': array([-1]), 'answer_text': a...","In the end, President Truman made the final de...",https://en.wikipedia.org/wiki/History%20of%20n...,[-1],[''],"['who', 'developed', 'the', 'first', 'thermonu...","['in', 'the', 'end', ',', 'president', 'truman...",[],0,0.285714


In [8]:
word_count.head()

,word,counters,norm,language
0,の,58790.0,0.000000,japanese
1,、,52354.0,0.109476,japanese
2,に,42765.0,0.272585,japanese
3,。,31939.0,0.456735,japanese
4,は,31421.0,0.465546,japanese


In [38]:
def get_highest_wordscore(que, language, data):
    count = word_count.loc[(word_count['word'].isin(eval(que)))&(word_count['language']==language)]
    return count.iloc[np.argmax(count['norm'])]['norm']
     

get_highest_wordscore(dft_eng.question_text_tokenized[0],dft_eng.language[0], word_count)

0.9996961121346224

In [20]:
dft_eng.question_text_tokenized[0]

"['when', 'was', 'quantum', 'field', 'theory', 'developed', '?']"

In [44]:
for df in [dft_jap,dfv_jap,dft_eng,dfv_eng,dft_fin,dfv_fin]:
    highest_wordscore = []
    for que,language in zip(df['question_text_tokenized'], df['language']):
        count = word_count.loc[(word_count['word'].isin(eval(que)))&(word_count['language']==language)]
        highest_wordscore.append(count.iloc[np.argmax(count['norm'])]['norm'])

    df['highest_wordscore'] = highest_wordscore
    

KeyboardInterrupt: 

In [41]:
df.loc[df['question_text']=='Where was Sorley MacLean born?'].document_plaintext.values

array(['In 1956, MacLean was offered the position of head teacher of Plockton High School in Wester Ross, not far from where his paternal grandmother\'s family had lived. It was a difficult assignment as the remote location was not attractive to teacher candidates, and MacLean frequently had to teach himself due to vacancies.[27] While at Plockton, he promoted the use of Scottish Gaelic in formal education and campaigned for a Highers exam for learners of Gaelic.[4]:127 Before 1968, there was no separate exam for Gaelic learners, who had to compete with native speakers if they took Gaelic Highers. MacLean felt that this unfair policy discouraged many students from studying Gaelic, although he encouraged his students to take the exam even if they were not native speakers.[5]:34[lower-roman 8] In 1966, he presented a paper to the Gaelic Society of Inverness outlining the practical issues in Gaelic education. MacLean pointed out that in continental Europe, it was not uncommon to study thr

In [ ]:
df